In [23]:
import heapq

In [24]:
class Node:
    def __init__(self, x,y, zone, end):
        self.x = x
        self.y = y
        self.zone = zone
        self.end = end
        self.start = False
        self.path = False
        self.cost = float('inf')
        self.prev = []
        self.next = None
        self.direction = None
        self.real_cost = float('inf')

    def __repr__(self):
        if self.end:
            return 'E'
        if self.start:
            return 'S'
        if self.path:
            return self.direction
        return '.'
    
    def __lt__(self, other):
        return self.cost <= other.cost

In [25]:
class Wall:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return "#"

class Reindeer:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.facing = ">"

    def __repr__(self):
        return "S"

In [26]:
def open_file(file):
    zone = []
    start = None
    end = None
    with open(file) as f:
        for y,line in enumerate(f.readlines()):
            zone.append([])
            for x,block in enumerate(line):
                if block == ".":
                    zone[y].append(Node(x,y, zone, False))
                elif block == "#":
                    zone[y].append(Wall(x,y))
                elif block == "S":
                    node = Node(x,y, zone, False)
                    zone[y].append(node)
                    start = node
                    start.cost = 0
                    start.direction = ">"
                    start.start = True
                elif block == "E":
                    node = Node(x,y, zone, True)
                    zone[y].append(node)
                    end = node
    return zone, start, end

In [27]:
def dijkstra(zone, start, end):
    open_list = []
    closed_list = []
    open_list.append(start)

    while open_list:
        current = open_list[0]
        for node in open_list:
            if node.cost < current.cost:
                current = node

        current.visited = True
        open_list.remove(current)
        closed_list.append(current)

        if current == end:
            path = []
            while current.prev:
                path.append(current)
                current = current.prev
                current.path = True
            return path

        for x,y,d in [(0,1,"v"), (0,-1,"^"), (1,0,">"), (-1,0,"<")]:
            node = zone[current.y + y][current.x + x]
            
            if isinstance(node, Wall):
                continue

            additional_cost = 0 if d == current.direction else 1000

            if node in closed_list:
                continue
            if node in open_list:
                if current.cost + 1 + additional_cost < node.cost:
                    node.cost = current.cost + 1 + additional_cost
                    node.prev = current
                    node.real_cost = 1 + additional_cost
            else:
                node.cost = current.cost + 1 + additional_cost
                node.prev = current
                open_list.append(node)
                node.real_cost = 1 + additional_cost
            node.direction = d

    return None        

In [28]:
def dijkstra_2(zone, start, end):
    open_list = [start]
    paths = []

    while open_list:
        current = heapq.heappop(open_list)

        if current == end:
            continue

        for x,y,d in [(0,1,"v"), (0,-1,"^"), (1,0,">"), (-1,0,"<")]:
            node = zone[current.y + y][current.x + x]
            
            if isinstance(node, Wall):
                continue

            additional_cost = 0 if d == current.direction else 1000

            new_cost = current.cost + 1 + additional_cost

            if new_cost <= node.cost:
                if new_cost < node.cost:
                    node.cost = new_cost
                    node.prev = []
                node.prev.append(current)
                node.real_cost = 1 + additional_cost
                node.direction = d
                heapq.heappush(open_list,node)

    def find_paths(node, path, paths):
        if node == start:
            paths.append(path[::-1])
            return

        for parent in node.prev:
            parent.path = True
            find_paths(parent, path + [node], paths)

    paths = []
    find_paths(end, [], paths)
    return paths      


# Example

In [29]:
zone, start, end = open_file("example_day16.txt")
for row in zone:
    for block in row:
        print(block, end="")
    print()

##########
#.......E#
#.##.#####
#..#.....#
##.#####.#
#S.......#
##########


In [30]:
path = dijkstra(zone,start,end)
sum([node.real_cost for node in path])

4019

In [31]:
for row in zone:
    for block in row:
        print(block, end="")
    print()

##########
#...^>>>E#
#.##^#####
#..#<<<<^#
##.#####^#
#S>>>>>>>#
##########


In [32]:
zone, start, end = open_file("example_day16.txt")
path = dijkstra_2(zone,start,end)
for row in zone:
    for block in row:
        print(block, end="")
    print()
print(len(path))

##########
#...^>>>E#
#.##^#####
#..#<<<<^#
##.#####^#
#S>>>>>>>#
##########
1


# Part 1

In [33]:
zone, start, end = open_file("data_day16.txt")
path = dijkstra(zone,start,end)
sum([node.real_cost for node in path])

114476